The idea is to match whatever procurer/supplier appears in the Dataset v5 file with an UBO, so the script should always looking for next owner in the book1 until it hits the UBO

In [2]:
### Works, but fix multiple owners issue somehow..

import pandas as pd
from tqdm import tqdm

# Read the Excel file with INN numbers of contracts
contracts_file_path = 'Dataset v5.xlsx'  # Replace with your file path
contracts_df = pd.read_excel(contracts_file_path)

# Read the Excel file with INN and respective owners
owners_file_path = 'Book1.xlsx'  # Replace with your file path
owners_df = pd.read_excel(owners_file_path)

# Create dictionaries to map INNs to their ultimate beneficiaries for supplier and procurer
supinn_to_owner = dict(zip(owners_df['INN'], owners_df['Ultimate Beneficiary']))
proinn_to_owner = dict(zip(owners_df['INN'], owners_df['Ultimate Beneficiary']))

# Function to get the ultimate beneficiary for an INN (Supplier)
def get_supplier_ultimate_beneficiary(supinn):
    current_inn = supinn
    max_iterations = 10  # Set a maximum number of iterations to avoid infinite loops
    iterations = 0
    ultimate_beneficiary = None  # Initialize the variable

    while current_inn in supinn_to_owner and iterations < max_iterations:
        ultimate_beneficiary = supinn_to_owner[current_inn]
        next_inn = owners_df.loc[owners_df['INN'] == current_inn, 'UBO'].iloc[0]

        # Print debug information
        #print(f"Supplier - Current INN: {current_inn}, Next INN: {next_inn}, Ultimate Beneficiary: {ultimate_beneficiary}")

        # Check if the next INN is a number or not
        if pd.notna(next_inn) and pd.to_numeric(next_inn, errors='coerce') == pd.to_numeric(next_inn, errors='coerce'):
            current_inn = next_inn
        else:
            return ultimate_beneficiary

        iterations += 1

    return ultimate_beneficiary

# Function to get the ultimate beneficiary for an INN (Procurer)
def get_procurer_ultimate_beneficiary(proinn):
    current_inn = proinn
    max_iterations = 10  # Set a maximum number of iterations to avoid infinite loops
    iterations = 0
    ultimate_beneficiary = None  # Initialize the variable

    while current_inn in proinn_to_owner and iterations < max_iterations:
        ultimate_beneficiary = proinn_to_owner[current_inn]
        next_inn = owners_df.loc[owners_df['INN'] == current_inn, 'UBO'].iloc[0]

        # Print debug information
        #print(f"Procurer - Current INN: {current_inn}, Next INN: {next_inn}, Ultimate Beneficiary: {ultimate_beneficiary}")

        # Check if the next INN is a number or not
        if pd.notna(next_inn) and pd.to_numeric(next_inn, errors='coerce') == pd.to_numeric(next_inn, errors='coerce'):
            current_inn = next_inn
        else:
            return ultimate_beneficiary

        iterations += 1

    return ultimate_beneficiary

# Apply the function to the first 1000 rows of the contracts file for 'supinn' and 'proinn'
tqdm.pandas(desc="Supplier Ultimate Beneficiary")
contracts_df['Supplier_Ultimate_Beneficiary'] = contracts_df['supinn'].progress_apply(get_supplier_ultimate_beneficiary)

tqdm.pandas(desc="Procurer Ultimate Beneficiary")
contracts_df['Procurer_Ultimate_Beneficiary'] = contracts_df['proinn'].progress_apply(get_procurer_ultimate_beneficiary)

# Save the modified contracts file with two columns for final beneficiaries
contracts_df.to_excel('Book3-Construction.xlsx', index=False)


Procurer Ultimate Beneficiary: 100%|██████████| 41098/41098 [00:52<00:00, 782.11it/s]
